# Latest Barstool Sports stories

#### Python tools and notebook settings

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import altair as alt
import numpy as np
import datetime as dt
from datetime import date
import re
import time
import random
import json

In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [4]:
month_year_updated = dt.date.today().strftime("%m_%Y")
today = dt.date.today().strftime("%Y-%m-%d")

---

In [5]:
headers = {
    "sec-ch-ua": '" Not A;Brand";v="99", "Chromium";v="101", "Google Chrome";v="101"',
    "Referer": "https://www.gunviolencearchive.org/",
    "sec-ch-ua-mobile": "?0",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.0.0 Safari/537.36",
    "sec-ch-ua-platform": '"macOS"',
}

#### Snag jsons using pagination in API

In [6]:
data = []

for r in range(1, 80):
    request = requests.get(
        f"https://union.barstoolsports.com/v2/stories/latest?&limit=4000&page={r}",
        headers=headers,
    )
    data.append(pd.DataFrame(request.json()))

In [7]:
stories_df = pd.concat(data)

In [8]:
stories_df.head(1)

,id,brand_id,title,type,url,branch_url,thumbnail,nsfw,comment_count,comment_status_open,category,tag,tags,author,post_type_meta,product,campaign,viewability,is_pinned,is_featured,is_published,is_hidden,is_show_guide,is_membership,is_archive,is_search_indexed,gated_product_id,access_level,ads_disabled,is_featured_section,blocker_description,is_instant_article,site_roughnrowdy,user_id,content_tags,author_is_active,brand,brand_name,is_gold,date,slug,hidden_tags,sites,urls,sharing,updated_at
0,3442295,NaN,Friendly Reminder: A Christmas Story Sucks,standard_post,https://www.barstoolsports.com/blog/3442295/friendly-reminder-a-christmas-story-sucks,https://bars.tl/3442295,"{'type': 'small', 'location': 'https://chumley.barstoolsports.com/union/giphy/2022/11/02/', 'images': {'small': '19307546.gif?crop=4%3A3', 'medium': '19307546.gif?crop=4%3A3', 'large': '19307546.gif?crop=4%3A3', 'featured': None}, 'desktop': 'https://chumley.barstoolsports.com/union/giphy/2022/11/02/19307546.gif?crop=4%3A3&width=560', 'raw': 'https://chumley.barstoolsports.com/union/giphy/2022/11/02/19307546.gif?crop=4%3A3', 'raw_desktop': 'https://chumley.barstoolsports.com/union/giphy/2022/11/02/19307546.gif?crop=4%3A3', 'featured': None, 'raw_featured': None}",False,0,True,"[{'id': '102', 'slug': 'barstoolu', 'name': 'BarstoolU'}]",None,"[movies, trailers, a-christmas-story]","{'id': 41523, 'name': 'Rico Bosco', 'author_url': '/bio/41523/rico-bosco/rico-bosco', 'avatar': 'https://chumley.barstoolsports.com/union/2020/09/29/Rico-Weather-1.0546e108.png?crop=320%3A320%2Csmart&width=64&height=64', 'twitter_handle': None, 'has_notifications': True, 'is_active': True}","{'standard_post': {}, 'barstool_original': None, 'podcast': None, 'gallery': {'images': [], 'id': '3442295', 'base': ''}, 'live': None, 'fundraiser': None}",{'id': None},None,0,0,0,1,0,0,0,0,1,None,0,0,0.0,,0,0,o51VAJhoe1rSVFCjUonWDD,"[{'metrics': {'published_posts': 2581, 'published_media': 3}, 'tag': 'movies', 'created_at': '2021-03-09T17:17:36.931Z', 'slug': 'movies', 'updated_at': '2022-11-02T16:34:45.727Z', 'visibility': 'public', 'id': 'jDZY2fbSTBXfRD878wz4zf'}, {'metrics': {'published_posts': 122}, 'tag': 'trailers', 'created_at': '2021-03-09T17:17:57.410Z', 'slug': 'trailers', 'updated_at': '2022-11-02T16:34:45.728Z', 'visibility': 'public', 'id': '7euxZf7bWjCP8GnqELtcCg'}, {'metrics': {'published_posts': 3}, 'tag': 'a christmas story', 'visibility': 'public', 'created_at': '2021-11-17T15:55:29.438Z', 'slug': 'a-christmas-story', 'updated_at': '2022-11-02T16:34:45.728Z', 'id': '67cba96e-dd4b-4ee6-9204-7ecfcfc414d8'}]",1.0,None,None,False,2022-11-02T16:40:00.000Z,friendly-reminder-a-christmas-story-sucks,[],"{'barstoolsports': True, 'barstoolbets': False, 'roughnrowdy': False}","{'barstoolsports': 'https://www.barstoolsports.com/blog/3442295/friendly-reminder-a-christmas-story-sucks', 'barstoolbets': 'https://www.barstoolbets.com/blogs/3442295/friendly-reminder-a-christmas-story-sucks', 'roughnrowdy': 'https://roughnrowdybrawl.com/blog/3442295/friendly-reminder-a-christmas-story-sucks', 'canonical': 'https://www.barstoolsports.com/blog/3442295/friendly-reminder-a-christmas-story-sucks'}","{'facebook': {'title': 'Friendly Reminder: A Christmas Story Sucks', 'url': 'https://bars.tl/3442295'}, 'twitter': {'title': 'Friendly Reminder: A Christmas Story Sucks', 'url': 'https://bars.tl/3442295'}, 'sms': {'title': 'Friendly Reminder: A Christmas Story Sucks', 'url': 'https://bars.tl/3442295'}, 'email': {'title': 'Friendly Reminder: A Christmas Story Sucks', 'url': 'https://bars.tl/3442295'}}",2022-11-02T16:41:04.000Z


In [9]:
stories_df["author_name"] = pd.json_normalize(stories_df["author"])["name"]

In [10]:
stories_df["author_id"] = pd.json_normalize(stories_df["author"])["id"]

In [11]:
stories_df["img_url"] = pd.json_normalize(stories_df["thumbnail"])["desktop"]

#### Slim down the frame

In [12]:
src = stories_df[
    [
        "id",
        "date",
        "type",
        "title",
        "author_name",
        "comment_count",
        "brand_name",
        "category",
        "branch_url",
        "img_url",
        "tags",
    ]
].copy()

In [13]:
src.head()

,id,date,type,title,author_name,comment_count,brand_name,category,branch_url,img_url,tags
0,3442295,2022-11-02T16:40:00.000Z,standard_post,Friendly Reminder: A Christmas Story Sucks,Rico Bosco,0,None,"[{'id': '102', 'slug': 'barstoolu', 'name': 'BarstoolU'}]",https://bars.tl/3442295,https://chumley.barstoolsports.com/union/giphy/2022/11/02/19307546.gif?crop=4%3A3&width=560,"[movies, trailers, a-christmas-story]"
1,3442210,2022-11-02T16:30:00.000Z,standard_post,"Apparently You Can Make Over $300,000 a Year From Just Owning Vending Machines",Marty Mush,0,None,"[{'id': '102', 'slug': 'barstoolu', 'name': 'BarstoolU'}]",https://bars.tl/3442210,https://chumley.barstoolsports.com/union/unsplash/2021/09/19/b94e8eca.jpeg?crop=4%3A3%2Csmart&width=560,[vending-machine]
2,3442290,2022-11-02T16:20:00.000Z,standard_post,"Awesome Video Of Cops Dressed As Spider-Man, Captain America, And Thor Making A Big Drug Bust During Halloween Weekend As The Avengers Music Blared Out",Clem,0,None,"[{'id': '98', 'slug': 'new-york', 'name': 'New York'}]",https://bars.tl/3442290,https://chumley.barstoolsports.com/union/2022/11/02/vi.1d98222d.png?crop=0.99%2C0.99%2Cx0.00%2Cy0.01%2Csafe&width=560,"[spider-man, superheroes, thor, halloween, drug-bust, captain-america]"
3,3442301,2022-11-02T16:10:00.000Z,standard_post,That'll Show Them - Kansas Suspends Bill Self For The First Four Games This Season Thanks To The 2017 FBI Investigation,Reags,0,None,"[{'id': '102', 'slug': 'barstoolu', 'name': 'BarstoolU'}]",https://bars.tl/3442301,https://chumley.barstoolsports.com/union/getty/2022/11/02/GettyImages-1389610324.33ef55b2.jpeg?crop=4%3A3%2Csmart&width=560,"[sports, college-basketball, kansas, bill-self, fbi-investigation]"
4,3442306,2022-11-02T16:00:00.000Z,barstool_original,Is It Ok For Men to Cry in Front of Women?,Eddie,2,The Dog Walk,"[{'id': '99', 'slug': 'chicago', 'name': 'Chicago'}]",https://bars.tl/3442306,https://chumley.barstoolsports.com/union/2022/11/02/Crying_BLOG_Thumbnail.2f922490.png?crop=4%3A3%2Csmart&width=560,"[the-dog-walk, dog-walk, crying, crying-in-front-of-women, men-crying-in-front-of-women, men-crying-women]"


#### Dates/times - convert to Eastern

In [14]:
src["published_date"] = pd.to_datetime(src["date"]).dt.tz_convert("US/Eastern").dt.date
src["published_hour"] = pd.to_datetime(src["date"]).dt.tz_convert("US/Eastern").dt.hour
src["published_day"] = (
    pd.to_datetime(src["date"]).dt.tz_convert("US/Eastern").dt.day_name()
)
src["timestamp"] = pd.to_datetime(src["date"]).dt.tz_convert("US/Eastern")
src.drop("date", axis=1, inplace=True)

In [15]:
src["week_start"] = (
    pd.to_datetime(src["published_date"])
    .dt.to_period("W")
    .apply(lambda r: r.start_time)
)

#### Tags and categories

In [16]:
src["first_tag"] = [l[0] if len(l) > 0 else np.nan for l in src["tags"]]

In [17]:
src[["category_id", "category_slug", "category_name"]] = pd.json_normalize(
    pd.json_normalize(src["category"])[0]
)

In [18]:
src.drop(["category", "category_id", "category_slug"], axis=1, inplace=True)

#### Filter the dataframe to just blog posts

In [19]:
df = src.copy()

In [20]:
df["published_date"].min()

datetime.date(2005, 3, 23)

In [21]:
len(df[df["type"] == "standard_post"])

193474

In [22]:
df.published_date.min()

datetime.date(2005, 3, 23)

---

#### Posts by Billy Football and Rico Bosco

In [23]:
standard_posts = df[df["type"] == "standard_post"]

In [24]:
bf_rb = standard_posts[
    standard_posts["author_name"].str.contains("Billy Football|Rico Bosco")
]

In [25]:
len(bf_rb)

5643

In [26]:
bf_rb[bf_rb["author_name"] == "Rico Bosco"]["published_date"].min()

datetime.date(2010, 10, 10)

In [27]:
bf_rb[bf_rb["author_name"] == "Billy Football"]["published_date"].min()

datetime.date(2013, 1, 3)

#### Post and comment totals for Billy v. Rico: June 2016-June 2022

In [28]:
bf_rb_grouped = (
    bf_rb.groupby(["author_name"])
    .agg({"id": "count", "comment_count": sum})
    .reset_index()
).rename(columns={"id": "posts"})

#### Comments per post

In [29]:
bf_rb_grouped["comments_per_post"] = (
    bf_rb_grouped["comment_count"] / bf_rb_grouped["posts"]
).round(2)

In [30]:
bf_rb_grouped

,author_name,posts,comment_count,comments_per_post
0,Billy Football,1929,30909,16.02
1,Rico Bosco,3714,56396,15.18


---

#### Posts by author

In [31]:
posts_grouped = (
    df[df["type"] == "standard_post"]
    .groupby("author_name")
    .agg({"comment_count": sum, "id": "count"})
    .reset_index()
    .rename(columns={"id": "post_count"})
)

In [32]:
posts_grouped["comments_per_post"] = (
    posts_grouped["comment_count"] / posts_grouped["post_count"]
).round(2)

In [33]:
posts_grouped_freq = posts_grouped[posts_grouped["post_count"] > 10].sort_values(
    "comments_per_post", ascending=False
)

In [34]:
posts_grouped_freq.sort_values("comments_per_post", ascending=False).head()

,author_name,comment_count,post_count,comments_per_post
2,Alex Bennett,1046,48,21.79
61,Kelly In Vegas,4363,240,18.18
36,Francis,12102,677,17.88
98,Tiko Texas,1794,103,17.42
64,Kim,2455,141,17.41


---

## Export

In [35]:
df.to_csv(f"/Users/stiles/data/barstool_{today}.csv", index=False)

In [36]:
df.to_csv("/Users/stiles/data/barstool.csv", index=False)

In [37]:
stories_df.to_json(
    f"/Users/stiles/data/barstool_{today}.json", indent=4, orient="records"
)